## Stabilization, P = 1.4

In [1]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

p_max = 1.4;
t_max = 5.0;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.0_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 7.0, tauP = 10.0, P = p_max);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [3]:
hoomd.run(1e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 756140 / 856000 | TPS 13.9461 | ETA 01:59:20
Time 00:00:20 | Step 756282 / 856000 | TPS 14.0643 | ETA 01:58:10
Time 00:00:30 | Step 756430 / 856000 | TPS 14.7606 | ETA 01:52:25
Time 00:00:40 | Step 756588 / 856000 | TPS 15.6681 | ETA 01:45:44
Time 00:00:50 | Step 756759 / 856000 | TPS 17.0913 | ETA 01:36:46
Time 00:01:00 | Step 756900 / 856000 | TPS 14.0743 | ETA 01:57:21
Time 00:01:10 | Step 757064 / 856000 | TPS 16.3644 | ETA 01:40:45
Time 00:01:20 | Step 757228 / 856000 | TPS 16.2958 | ETA 01:41:01
Time 00:01:30 | Step 757388 / 856000 | TPS 15.954 | ETA 01:43:01
Time 00:01:40 | Step 757559 / 856000 | TPS 16.9377 | ETA 01:36:51
Time 00:01:50 | Step 757744 / 856000 | TPS 18.3509 | ETA 01:29:14
Time 00:02:00 | Step 757930 / 85

In [4]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x111c879b0>),
             ('particles', <hoomd.data.particle_data at 0x111badf28>),
             ('number_density', 0.7820120350820375),
             ('bonds', <hoomd.data.bond_data at 0x111badf98>),
             ('angles', <hoomd.data.angle_data at 0x111c87080>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x111c870f0>),
             ('impropers', <hoomd.data.dihedral_data at 0x111c87160>),
             ('constraints', <hoomd.data.constraint_data at 0x111c871d0>),
             ('pairs', <hoomd.data.bond_data at 0x111c87240>),
             ('timestep', 856000)])